# SimFin Tutorial 10 - Neural Networks

[Original repository on GitHub](https://github.com/simfin/simfin-tutorials)

This tutorial was originally written by [Hvass Labs](https://github.com/Hvass-Labs)

----

"The attackers can be stopped by removing the head or destroying the brain." &ndash; [Shaun of The Dead](https://www.youtube.com/watch?v=ONEZfj6h1xc)

## Introduction

In the previous tutorial we used a Machine Learning (ML) model to try and predict future stock-returns for 1-3 year investment periods given various signals. This worked quite well on the training-set, but unfortunately it failed to generalize to the test-set.

In this tutorial we will use a so-called Neural Network, which has become very fashionable in recent years, because they have been able to solve extremely hard problems, that were previously impossible to solve.

Unfortunately, Neural Networks have become so hyped, that people sometimes misunderstand what they really are: A Neural Network is just a function approximator that maps some input x (e.g. signals) to some output y (e.g. stock-returns). It consists of a number of layers of both linear and non-linear mappings, whose parameters will be learned during the training process, so it learns to map from x to y in the training-set, and the learned mapping hopefully generalizes to new data.

In theory, a Neural Network is capable of approximating any function to any desired degree of precision - but what is truly important, is not how well it performs on the training-set, but how well it performs on new data that it has not seen during the training.

This tutorial shows how to use financial data from SimFin with the TensorFlow library for creating Neural Networks. It is assumed you are already familiar with both.

## Imports

In [19]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns

# TensorFlow and Keras imports.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.utils import to_categorical

# Scikit-learn imports.
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

# Import the main functionality from the SimFin Python API.
import simfin as sf

# Import names used for easy access to SimFin's data-columns.
from simfin.names import *

In [20]:
import tensorflow.keras as keras

In [21]:
# Version of the SimFin Python API.
sf.__version__

'0.8.3'

In [22]:
# TensorFlow version.
tf.__version__

'2.6.0'

## SimFin Config

In [23]:
# SimFin data-directory.
sf.set_data_dir('~/simfin_data/')

In [24]:
# SimFin load API key or use free data.
sf.load_api_key(path='~/simfin_api_key.txt', default_key='free')

In [25]:
# Seaborn set plotting style.
sns.set_style("whitegrid")

## Data Hub

We use a `StockHub` object to easily load and process financial data with these settings:

In [26]:
# We are interested in the US stock-market.
market = 'us'

# Add this date-offset to the fundamental data such as
# Income Statements etc., because the REPORT_DATE is not
# when it was actually made available to the public,
# which can be 1, 2 or even 3 months after the Report Date.
offset = pd.DateOffset(days=60)

# Refresh the fundamental datasets (Income Statements etc.)
# every 30 days.
refresh_days = 30

# Refresh the dataset with shareprices every 10 days.
refresh_days_shareprices = 10

In [27]:
%%time
hub = sf.StockHub(market=market, offset=offset,
                  refresh_days=refresh_days,
                  refresh_days_shareprices=refresh_days_shareprices)

CPU times: user 16 µs, sys: 6 µs, total: 22 µs
Wall time: 28.4 µs


## Signals

First we calculate financial signals for the stocks, such as the Current Ratio, Debt Ratio, Net Profit Margin, Return on Assets, etc. These are calculated using data from the financial reports: Income Statements, Balance Sheets and Cash-Flow Statements, which are automatically downloaded and loaded by the data-hub.

In [28]:
%%time
df_fin_signals = hub.fin_signals(variant='daily')

Dataset "us-income-ttm" on disk (8 days old).
- Loading from disk ... Done!
Dataset "us-balance-ttm" on disk (8 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-ttm" on disk (8 days old).
- Loading from disk ... Done!
Dataset "us-shareprices-daily" on disk (8 days old).
- Loading from disk ... Done!
Cache-file 'fin_signals-2a38bb7d.pickle' on disk (8 days old).
- Loading from disk ... Done!
CPU times: user 6.7 s, sys: 1.17 s, total: 7.87 s
Wall time: 7.9 s


Now we calculate growth signals for the stocks, such as Earnings Growth, FCF Growth, Sales Growth, etc. These are also calculated using data from the financial reports: Income Statements, Balance Sheets and Cash-Flow Statements, which are automatically downloaded and loaded by the data-hub.

In [29]:
%%time
df_growth_signals = hub.growth_signals(variant='daily')

Dataset "us-income-quarterly" on disk (8 days old).
- Loading from disk ... Done!
Dataset "us-balance-quarterly" on disk (8 days old).
- Loading from disk ... Done!
Dataset "us-cashflow-quarterly" on disk (8 days old).
- Loading from disk ... Done!
Cache-file 'growth_signals-2a38bb7d.pickle' on disk (8 days old).
- Loading from disk ... Done!
CPU times: user 729 ms, sys: 266 ms, total: 994 ms
Wall time: 1.01 s


In [30]:
df_growth_signals

Assets Growth  Assets Growth QOQ  Assets Growth YOY  \
Ticker Date                                                              
A      2007-01-03            NaN                NaN                NaN   
       2007-01-04            NaN                NaN                NaN   
       2007-01-05            NaN                NaN                NaN   
       2007-01-08            NaN                NaN                NaN   
       2007-01-09            NaN                NaN                NaN   
...                          ...                ...                ...   
ZYXI   2020-09-14       0.683181           0.106466           0.683181   
       2020-09-15       0.683181           0.106466           0.683181   
       2020-09-16       0.683181           0.106466           0.683181   
       2020-09-17       0.683181           0.106466           0.683181   
       2020-09-18       0.683181           0.106466           0.683181   

                   Earnings Growth  Earnings Growth QOQ  Earnings Growth YOY  \
Ticker Date                                                                    
A      2007-01-03              NaN                  NaN                  NaN   
       2007-01-04              NaN                  NaN                  NaN   
       2007-01-05              NaN                  NaN                  NaN   
       2007-01-08              NaN                  NaN                  NaN   
       2007-01-09              NaN                  NaN                  NaN   
...                            ...                  ...                  ...   
ZYXI   2020-09-14         0.124203             0.027239             0.395467   
       2020-09-15         0.124203             0.027239             0.395467   
       2020-09-16         0.124203             0.027239             0.395467   
       2020-09-17         0.124203             0.027239             0.395467   
       2020-09-18         0.124203             0.027239             0.395467   

                   FCF Growth  FCF Growth QOQ  FCF Growth YOY  Sales Growth  \
Ticker Date                                                                   
A      2007-01-03         NaN             NaN             NaN           NaN   
       2007-01-04         NaN             NaN             NaN           NaN   
       2007-01-05         NaN             NaN             NaN           NaN   
       2007-01-08         NaN             NaN             NaN           NaN   
       2007-01-09         NaN             NaN             NaN           NaN   
...                       ...             ...             ...           ...   
ZYXI   2020-09-14   -0.212002        5.847761        2.823333      0.636049   
       2020-09-15   -0.212002        5.847761        2.823333      0.636049   
       2020-09-16   -0.212002        5.847761        2.823333      0.636049   
       2020-09-17   -0.212002        5.847761        2.823333      0.636049   
       2020-09-18   -0.212002        5.847761        2.823333      0.636049   

                   Sales Growth QOQ  Sales Growth YOY  
Ticker Date                                            
A      2007-01-03               NaN               NaN  
       2007-01-04               NaN               NaN  
       2007-01-05               NaN               NaN  
       2007-01-08               NaN               NaN  
       2007-01-09               NaN               NaN  
...                             ...               ...  
ZYXI   2020-09-14          0.264972          0.870739  
       2020-09-15          0.264972          0.870739  
       2020-09-16          0.264972          0.870739  
       2020-09-17          0.264972          0.870739  
       2020-09-18          0.264972          0.870739  

[7113810 rows x 12 columns]

Now we calculate valuation signals for the stocks, such as P/E, P/Sales, etc. These are calculated from the share-prices and data from the financial reports. Because the data-hub has already loaded the required datasets in the function-calls above, the data is merely reused here, and the data-hub can proceed directly to computing the signals.

In [31]:
%%time
df_val_signals = hub.val_signals(variant='daily')

Cache-file 'val_signals-739b68a6.pickle' on disk (8 days old).
- Loading from disk ... Done!
CPU times: user 31.2 ms, sys: 230 ms, total: 261 ms
Wall time: 263 ms


We now combine all the signals into a single DataFrame:

In [32]:
%%time
# Combine the DataFrames.
dfs = [df_fin_signals, df_growth_signals, df_val_signals]
df_signals = pd.concat(dfs, axis=1)

CPU times: user 2.88 s, sys: 555 ms, total: 3.43 s
Wall time: 3.47 s


We can then show the results:

In [33]:
df_signals.dropna(how='all').head()

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2010-09-29                           0.171492        0.553297   
       2010-09-30                           0.171492        0.553297   
       2010-10-01                           0.171492        0.553297   
       2010-10-04                           0.171492        0.553297   
       2010-10-05                           0.171492        0.553297   

                   CapEx / (Depr + Amor)  Current Ratio  Debt Ratio  \
Ticker Date                                                           
A      2010-09-29               0.622857       1.966061    0.404176   
       2010-09-30               0.622857       1.966061    0.404176   
       2010-10-01               0.622857       1.966061    0.404176   
       2010-10-04               0.622857       1.966061    0.404176   
       2010-10-05               0.622857       1.966061    0.404176   

                   Dividends / FCF  Gross Profit Margin  Interest Coverage  \
Ticker Date                                                                  
A      2010-09-29             -0.0             0.542205           5.636364   
       2010-09-30             -0.0             0.542205           5.636364   
       2010-10-01             -0.0             0.542205           5.636364   
       2010-10-04             -0.0             0.542205           5.636364   
       2010-10-05             -0.0             0.542205           5.636364   

                   Inventory Turnover  Log Revenue  ...  Earnings Yield  \
Ticker Date                                         ...                   
A      2010-09-29            7.318314     9.701999  ...        0.036491   
       2010-09-30            7.318314     9.701999  ...        0.035627   
       2010-10-01            7.318314     9.701999  ...        0.035247   
       2010-10-04            7.318314     9.701999  ...        0.036480   
       2010-10-05            7.318314     9.701999  ...        0.035983   

                   FCF Yield    Market-Cap    P/Cash        P/E      P/FCF  \
Ticker Date                                                                  
A      2010-09-29   0.039291  1.142744e+10  4.931996  27.403921  25.450857   
       2010-09-30   0.038361  1.170453e+10  5.051587  28.068411  26.067990   
       2010-10-01   0.037952  1.183080e+10  5.106084  28.371217  26.349215   
       2010-10-04   0.039279  1.143094e+10  4.933510  27.412332  25.458669   
       2010-10-05   0.038744  1.158878e+10  5.001631  27.790839  25.810200   

                      P/NCAV  P/NetNet   P/Sales  Price to Book Value  
Ticker Date                                                            
A      2010-09-29 -20.589973 -3.763357  2.269600             4.066703  
       2010-09-30 -21.089239 -3.854611  2.324633             4.165312  
       2010-10-01 -21.316752 -3.896195  2.349712             4.210248  
       2010-10-04 -20.596293 -3.764513  2.270296             4.067951  
       2010-10-05 -20.880685 -3.816493  2.301644             4.124121  

[5 rows x 42 columns]

## Missing Data

Some of the signals have a lot of missing data which cannot be handled by TensorFlow. Let us first see the fraction of each signal-column that is missing:

In [34]:
# Remove all rows with only NaN values.
df = df_signals.dropna(how='all').reset_index(drop=True)

# For each column, show the fraction of the rows that are NaN.
(df.isnull().sum() / len(df)).sort_values(ascending=False)

R&D / Gross Profit                   0.649276
Return on Research Capital           0.649276
R&D / Revenue                        0.614202
Dividend Yield                       0.438901
Inventory Turnover                   0.290166
Debt Ratio                           0.262063
Net Acquisitions / Total Assets      0.257896
Sales Growth                         0.147292
Sales Growth YOY                     0.144781
FCF Growth                           0.135264
Earnings Growth                      0.135264
Assets Growth                        0.135264
FCF Growth YOY                       0.133461
Earnings Growth YOY                  0.133446
Assets Growth YOY                    0.133446
Interest Coverage                    0.097272
Gross Profit Margin                  0.094301
Sales Growth QOQ                     0.053825
Assets Growth QOQ                    0.040359
Earnings Growth QOQ                  0.040359
FCF Growth QOQ                       0.040359
CapEx / (Depr + Amor)             

In [35]:
df

,(Dividends + Share Buyback) / FCF,Asset Turnover,CapEx / (Depr + Amor),Current Ratio,Debt Ratio,Dividends / FCF,Gross Profit Margin,Interest Coverage,Inventory Turnover,Log Revenue,...,Earnings Yield,FCF Yield,Market-Cap,P/Cash,P/E,P/FCF,P/NCAV,P/NetNet,P/Sales,Price to Book Value
0,0.171492,0.553297,0.622857,1.966061,0.404176,-0.000000,0.542205,5.636364,7.318314,9.701999,...,0.036491,0.039291,1.142744e+10,4.931996,27.403921,25.450857,-20.589973,-3.763357,2.269600,4.066703
1,0.171492,0.553297,0.622857,1.966061,0.404176,-0.000000,0.542205,5.636364,7.318314,9.701999,...,0.035627,0.038361,1.170453e+10,5.051587,28.068411,26.067990,-21.089239,-3.854611,2.324633,4.165312
2,0.171492,0.553297,0.622857,1.966061,0.404176,-0.000000,0.542205,5.636364,7.318314,9.701999,...,0.035247,0.037952,1.183080e+10,5.106084,28.371217,26.349215,-21.316752,-3.896195,2.349712,4.210248
3,0.171492,0.553297,0.622857,1.966061,0.404176,-0.000000,0.542205,5.636364,7.318314,9.701999,...,0.036480,0.039279,1.143094e+10,4.933510,27.412332,25.458669,-20.596293,-3.764513,2.270296,4.067951
4,0.171492,0.553297,0.622857,1.966061,0.404176,-0.000000,0.542205,5.636364,7.318314,9.701999,...,0.035983,0.038744,1.158878e+10,5.001631,27.790839,25.810200,-20.880685,-3.816493,2.301644,4.124121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4265802,-0.055400,1.645039,NaN,4.955648,NaN,0.000468,0.792558,912.071429,12.846824,7.781540,...,0.019939,0.011686,5.483703e+08,32.417254,50.152759,85.575885,27.376081,36.765128,9.068468,20.342407
4265803,-0.055400,1.645039,NaN,4.955648,NaN,0.000468,0.792558,912.071429,12.846824,7.781540,...,0.020001,0.011722,5.466598e+08,32.316140,49.996325,85.308961,27.290690,36.650452,9.040182,20.278956
4265804,-0.055400,1.645039,NaN,4.955648,NaN,0.000468,0.792558,912.071429,12.846824,7.781540,...,0.019964,0.011700,5.476861e+08,32.376808,50.090186,85.469115,27.341925,36.719258,9.057154,20.317027
4265805,-0.055400,1.645039,NaN,4.955648,NaN,0.000468,0.792558,912.071429,12.846824,7.781540,...,0.018561,0.010878,5.890790e+08,34.823775,53.875890,91.928680,29.408366,39.494417,9.741673,21.852542


Let us remove all signals that have more than 25% missing data:

In [36]:
# List of the columns before removing any.
columns_before = df_signals.columns

# Threshold for the number of rows that must be NaN for each column.
thresh = 0.75 * len(df_signals.dropna(how='all'))

# Remove all columns which don't have sufficient data.
df_signals = df_signals.dropna(axis='columns', thresh=thresh)

# List of the columns after the removal.
columns_after = df_signals.columns

# Show the columns that were removed.
columns_before.difference(columns_after)

Index(['Debt Ratio', 'Dividend Yield', 'Inventory Turnover',
       'Net Acquisitions / Total Assets', 'R&D / Gross Profit',
       'R&D / Revenue', 'Return on Research Capital'],
      dtype='object')

In [37]:
# print all columns
with pd.option_context('display.max_rows', 6, 'display.max_columns', None): 
    display(df_signals)

(Dividends + Share Buyback) / FCF  Asset Turnover  \
Ticker Date                                                            
A      2007-01-03                                NaN             NaN   
       2007-01-04                                NaN             NaN   
       2007-01-05                                NaN             NaN   
...                                              ...             ...   
ZYXI   2020-09-16                            -0.0554        1.645039   
       2020-09-17                            -0.0554        1.645039   
       2020-09-18                            -0.0554        1.645039   

                   CapEx / (Depr + Amor)  Current Ratio  Dividends / FCF  \
Ticker Date                                                                
A      2007-01-03                    NaN            NaN              NaN   
       2007-01-04                    NaN            NaN              NaN   
       2007-01-05                    NaN            NaN              NaN   
...                                  ...            ...              ...   
ZYXI   2020-09-16                    NaN       4.955648         0.000468   
       2020-09-17                    NaN       4.955648         0.000468   
       2020-09-18                    NaN       4.955648         0.000468   

                   Gross Profit Margin  Interest Coverage  Log Revenue  \
Ticker Date                                                              
A      2007-01-03                  NaN                NaN          NaN   
       2007-01-04                  NaN                NaN          NaN   
       2007-01-05                  NaN                NaN          NaN   
...                                ...                ...          ...   
ZYXI   2020-09-16             0.792558         912.071429      7.78154   
       2020-09-17             0.792558         912.071429      7.78154   
       2020-09-18             0.792558         912.071429      7.78154   

                   Net Profit Margin  Quick Ratio  Return on Assets  \
Ticker Date                                                           
A      2007-01-03                NaN          NaN               NaN   
       2007-01-04                NaN          NaN               NaN   
       2007-01-05                NaN          NaN               NaN   
...                              ...          ...               ...   
ZYXI   2020-09-16           0.180817     4.016611          0.297451   
       2020-09-17           0.180817     4.016611          0.297451   
       2020-09-18           0.180817     4.016611          0.297451   

                   Return on Equity  Share Buyback / FCF  Assets Growth  \
Ticker Date                                                               
A      2007-01-03               NaN                  NaN            NaN   
       2007-01-04               NaN                  NaN            NaN   
       2007-01-05               NaN                  NaN            NaN   
...                             ...                  ...            ...   
ZYXI   2020-09-16          0.405609            -0.055868       0.683181   
       2020-09-17          0.405609            -0.055868       0.683181   
       2020-09-18          0.405609            -0.055868       0.683181   

                   Assets Growth QOQ  Assets Growth YOY  Earnings Growth  \
Ticker Date                                                                
A      2007-01-03                NaN                NaN              NaN   
       2007-01-04                NaN                NaN              NaN   
       2007-01-05                NaN                NaN              NaN   
...                              ...                ...              ...   
ZYXI   2020-09-16           0.106466           0.683181         0.124203   
       2020-09-17           0.106466           0.683181         0.124203   
       2020-09-18           0.106466           0.683181         0.124203   

                 

## 1-3 Year Returns

We want to try and predict the average 1-3 year returns. We will actually consider the mean log-returns, because that is easier to calculate, but the result is nearly the same as the non-log mean returns.

In [ ]:
# Name of the new column for the returns.
TOTAL_RETURN_1_3Y = 'Total Return 1-3 Years'

In [ ]:
%%time
# Calculate the mean log-returns for all 1-3 year periods.
df_returns_1_3y = \
    hub.mean_log_returns(name=TOTAL_RETURN_1_3Y,
                         future=True, annualized=True,
                         min_years=1, max_years=3)

## 1 Year Returns

In [38]:
%%time
TOTAL_RETURN_1_1Y = 'Total Return 1-1 Years'
# Calculate the mean log-returns for only 1 year periods.
df_returns_1_1y = \
    hub.mean_log_returns(name=TOTAL_RETURN_1_1Y,
                         future=True, annualized=True,
                         min_years=0.5, max_years=1)

Cache-file 'mean_log_change-0cd85b4d.pickle' on disk (0 days old).
- Loading from disk ... Done!
CPU times: user 87 ms, sys: 156 ms, total: 243 ms
Wall time: 1.15 s


In [39]:
df_returns_1_1y

Ticker  Date      
A       2007-01-03    0.103831
        2007-01-04    0.097915
        2007-01-05    0.109227
        2007-01-08    0.112659
        2007-01-09    0.109247
                        ...   
ZYXI    2020-09-14         NaN
        2020-09-15         NaN
        2020-09-16         NaN
        2020-09-17         NaN
        2020-09-18         NaN
Name: Total Return 1-1 Years, Length: 7113810, dtype: float64

## price next year

In [50]:
df_signals.index

MultiIndex([(   'A', '2007-01-03'),
            (   'A', '2007-01-04'),
            (   'A', '2007-01-05'),
            (   'A', '2007-01-08'),
            (   'A', '2007-01-09'),
            (   'A', '2007-01-10'),
            (   'A', '2007-01-11'),
            (   'A', '2007-01-12'),
            (   'A', '2007-01-16'),
            (   'A', '2007-01-17'),
            ...
            ('ZYXI', '2020-09-04'),
            ('ZYXI', '2020-09-08'),
            ('ZYXI', '2020-09-09'),
            ('ZYXI', '2020-09-10'),
            ('ZYXI', '2020-09-11'),
            ('ZYXI', '2020-09-14'),
            ('ZYXI', '2020-09-15'),
            ('ZYXI', '2020-09-16'),
            ('ZYXI', '2020-09-17'),
            ('ZYXI', '2020-09-18')],
           names=['Ticker', 'Date'], length=7113810)

In [43]:
df_signals.index
df_signals['Date'] = pd.to_datetime(df_signals['Date'],format='%Y%m%d')
df_signals['Year'] = pd.DatetimeIndex(df_signals['Date']).year
df_signals.groupby(by=["Ticker","Year"], dropna=False).max()

KeyError: 'Date'

In [ ]:
price_1y

## Combine Signals & Returns

We then combine the signals and returns into a single DataFrame to align the data.

In [ ]:
%%time
dfs = [df_signals, df_returns_1_3y]
df_sig_rets = pd.concat(dfs, axis=1)

In [ ]:
%%time
dfs = [df_signals, df_returns_1_1y]
df_sig_rets = pd.concat(dfs, axis=1)

In [ ]:
dfs

In [ ]:
# print all columns
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(df_sig_rets)

## Data Cleaning

The data also contains outliers and still has missing values, so let us fix that now. A common method for removing outliers is so-called *Winsorization* of the data. It basically just limits or clips the data between e.g. the 5% and 95% quantiles of the data. We will Winsorize both the stock-returns and all the signals. Then we remove all rows with missing values, and finally we remove tickers that have less than 200 rows of data.

There are more than 5 million data-rows before this cleaning:

In [ ]:
df_sig_rets.shape

In [ ]:
df_signals.shape
df_sig_rets['Total Return 1-3 Years']
print(df_sig_rets.columns.tolist())

In [ ]:
df_signals.shape
df_sig_rets['Total Return 1-1 Years']
print(df_sig_rets.columns.tolist())

In [ ]:
%%time

# Clip both the signals and returns at their 5% and 95% quantiles.
# We do not set them to NaN because it would remove too much data.
df_sig_rets = sf.winsorize(df_sig_rets)

# Remove all rows with missing values (NaN)
# because TensorFlow cannot handle that.
df_sig_rets = df_sig_rets.dropna(how='any')

# Remove all tickers which have less than 200 data-rows.
df_sig_rets = df_sig_rets.groupby(TICKER) \
                .filter(lambda df: len(df)>200)

This data-cleaning has removed about 80% of the rows, which is a lot:

In [ ]:
df_sig_rets.shape

In [ ]:
df_sig_rets.describe()

In [ ]:
# print all columns
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(df_sig_rets)

In [ ]:
df_sig_rets.index

In [ ]:
# filter by one of the index
x=df_sig_rets[df_sig_rets.index.get_level_values('Ticker').isin(['IBM'])].sample(10)
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(x)

In [ ]:
with pd.option_context('display.max_rows', 10, 'display.max_columns', None): 
    display(df_sig_rets.loc['A'].sample(10))


The reason that so many rows have been removed, is that TensorFlow cannot handle rows with missing values (NaN). Some missing values could be set to a default value, e.g. the Dividend Yield could be set to 0. Other columns could be omitted because they don't have any predictive power anyway.

## Split Training / Test-Sets

When training a Machine Learning model, we typically split the dataset into training- and test-sets. Only the training-set is used to train the model, after which the model is used on the test-set, to assess how the model performs on data it has not seen during training.

If we randomly divide all the data-rows into training- and test-sets, we would most likely get data for every stock-ticker in both the training- and test-sets, which would probably make them highly correlated. To avoid this, we split the dataset according to stock-tickers, so a ticker belongs to either the training- or test-set, but not both. We use 80% of all the tickers in the training-set, and 20% in the test-set.

In [ ]:
# List of all unique stock-tickers in the dataset.
tickers = df_sig_rets.reset_index()[TICKER].unique()

# Split the tickers into training- and test-sets.
tickers_train, tickers_test = \
    train_test_split(tickers, train_size=0.8, random_state=1234)

In [ ]:
tickers_train, tickers_test

Now that we have lists of tickers for the training- and test-sets, we can select those rows from the DataFrame with signals and stock-returns:

In [ ]:
%%time
df_train = df_sig_rets.loc[tickers_train]
df_test = df_sig_rets.loc[tickers_test]

We can then split these DataFrames into signals X and stock-returns y, that the Machine Learning model must try and find a mapping between. We use a common notation where the capital X indicates a 2-dimensional array or matrix, and the lower-case y indicates a 1-dimensional array.

In [ ]:
# DataFrames with signals for training- and test-sets.
X_train = df_train.drop(columns=[TOTAL_RETURN_1_3Y])
X_test = df_test.drop(columns=[TOTAL_RETURN_1_3Y])

# DataFrames with returns for training- and test-sets.
y_train = df_train[TOTAL_RETURN_1_3Y]
y_test = df_test[TOTAL_RETURN_1_3Y]

In [ ]:
# DataFrames with signals for training- and test-sets.
X_train = df_train.drop(columns=[TOTAL_RETURN_1_1Y])
X_test = df_test.drop(columns=[TOTAL_RETURN_1_1Y])

# DataFrames with returns for training- and test-sets.
y_train = df_train[TOTAL_RETURN_1_1Y]
y_test = df_test[TOTAL_RETURN_1_1Y]

Let us check that the shapes of these arrays look reasonable:

In [ ]:
X_train.shape, y_train.shape

In [ ]:
X_test.shape, y_test.shape

Let us also get the number of signals, which will be convenient later:

In [ ]:
num_signals = X_train.shape[1]
num_signals

## Scale Signals

Neural Networks require that the input has "moderate" values e.g. between 0 and 1, or mean 0 and 1 standard deviation, which is the method we use here. It is not necessary to scale the stock-returns because they have already been Winsorized to remove outliers, and they are centered around 0 with a small standard deviation.

If you want to input new data to the Neural Network, then you have to run the signals through the scaler first, because the Neural Network has been tuned to the scaled signals.

In [ ]:
# Create a scaler and fit it to the signals in the training-data.
signal_scaler = StandardScaler()
signal_scaler.fit(X_train)

def scale_signal(df):
    """Apply the signal-scaler to the given DataFrame."""

    # Apply the scaler. This returns a numpy array.
    array = signal_scaler.transform(df)
    
    # Create a DataFrame with the correct column-names and index.
    df_scaled = pd.DataFrame(data=array,
                             columns=df.columns,
                             index=df.index)
    return df_scaled

# Scale the signals in the training- and test-sets.
X_train = scale_signal(X_train)
X_test = scale_signal(X_test)

Except for small rounding errors, the signals in the training-set now have 0 mean and 1 standard deviation:

In [ ]:
X_train.describe()

Using the same scaler on the test-data, also gives means that are close to 0 and standard deviations close to 1:

In [ ]:
X_test.describe()

## Neural Network: Regression

Let us now build a Neural Network to learn the mapping from signals X to stock-returns y. We will use Keras and TensorFlow. If you are new to this, you can watch [this tutorial](https://www.youtube.com/watch?v=3yfRJKA1BiQ) for a basic explanation.

In [ ]:
# This so-called "activation function" is applied after each layer.
# It is important to use a non-linear function, otherwise
# the Neural Network cannot learn non-linear mappings.
activation = 'relu'

In [ ]:
# Create a new Keras model.
model_regr = Sequential()

# Add an input layer for the signals.
# Note how we set the dimensionality of the input here.
model_regr.add(InputLayer(input_shape=(num_signals,)))

# Add several dense aka. fully-connected layers.
# You can experiment with different designs.
model_regr.add(Dense(128, activation=activation))
model_regr.add(Dense(64, activation=activation))
model_regr.add(Dense(32, activation=activation))
model_regr.add(Dense(16, activation=activation))
model_regr.add(Dense(8, activation=activation))

# Add a layer for the output of the Neural Network.
# This is 1-dimensional to match the stock-return data.
model_regr.add(Dense(1))

# Compile the model but don't train it yet.
model_regr.compile(loss='mse', metrics=['mae'],
                   optimizer=RMSprop(0.001))

# Show the model.
model_regr.summary()

These are common arguments for training both our regression and classification models:

In [ ]:
fit_args = \
{
    # For efficiency, the model is trained on batches of data.
    'batch_size': 4096,
    
    # Number of iterations aka. epochs over the training-set.
    'epochs': 40,
    
    # Fraction of the training-set used for validation after
    # each training-epoch, to assess how well the model performs
    # on unseen data.
    'validation_split': 0.2,

    # Show status during training.
    'verbose': 1,
}

We can now train the model. This takes a minute or two on an ordinary computer:

In [ ]:
%%time
history_regr = model_regr.fit(x=X_train.values,
                              y=y_train.values, **fit_args)

We can then plot the training-progress as the Mean Absolute Error (MAE) which is supposed to be minimized. After 40 epochs through the training-set, the model has learned to map from signals to stock-returns for the training-set, with an average absolute error of about 6%, which would be very good if it generalized to unseen data. Unfortunately, the performance on the validation-set gets worse during optimization and ends with a MAE about 24%, so the model is clearly not learning the underlying relation between signals and stock-returns, but it has instead learned peculiarities and noise in the training-data.

In [ ]:
# Convert training history to a DataFrame.
df = pd.DataFrame(history_regr.history)

# Rename the relevant columns.
names = {'mae': 'Training MAE', 'val_mae': 'Validation MAE'}
df = df[['mae', 'val_mae']].rename(columns=names)

# Plot the results.
df.plot()

### Regression Results

Let us now compute some performance measures for the Neural Network. First we use the fitted model to predict the stock-returns for the training- and test-sets:

In [ ]:
%%time
y_train_pred = model_regr.predict(X_train.values)
y_test_pred = model_regr.predict(X_test.values)

We can then calculate the $R^2$ value (aka. the coefficient of determination) between the actual stock-returns and those predicted by the model. A value of 1 means a perfect prediction, while a value of 0 means the model basically predicted the average stock-return, and a negative value means the model's prediction was worse than just using the average stock-return.

The $R^2$ value is about 0.5 for the training-set, which is OK but could be much higher if we trained it more, and if we used a bigger Neural Network with more parameters to optimize.

In [ ]:
r2_score(y_true=y_train, y_pred=y_train_pred)

But the $R^2$ value is very negative for the test-set, which is so bad, that it would be much better to just use the average stock-return. This shows the model has not learned the underlying relation between the signals X and stock-returns y. The model has merely learned noise or peculiarities in the training-data, which do not generalize to the unseen data in the test-set:

In [ ]:
r2_score(y_true=y_test, y_pred=y_test_pred)

### Regression Examples

We can also try and plot the model's predicted versus actual stock-returns for a stock in the training-set. As we can see, the model has learned this mapping reasonably well:

In [ ]:
# Column-name for the models' predicted stock-returns.
TOTAL_RETURN_PRED = 'Total Return Predicted'

# Create a DataFrame with actual and predicted stock-returns.
# This is for the training-set.
df_y_train = pd.DataFrame(y_train)
df_y_train[TOTAL_RETURN_PRED] = y_train_pred

# Plot the actual and predicted stock-returns for the first stock.
ticker = tickers_train[0]
_ = df_y_train.loc[ticker].plot(title=ticker)

Now let us try and plot the model's predicted versus actual stock-returns for the first stock in the test-set. As we can see, the model is mostly unable to predict the stock-returns, for this data that it has not seen during its training:

In [ ]:
# Create a DataFrame with actual and predicted stock-returns.
# This is for the test-set.
df_y_test = pd.DataFrame(y_test)
df_y_test[TOTAL_RETURN_PRED] = y_test_pred

# Plot the actual and predicted stock-returns for the first stock.
ticker = tickers_test[0]
_ = df_y_test.loc[ticker].plot(title=ticker)

## Neural Network: Classification

Instead of mapping from signals X to a continuous variable y for the stock-return, we can try and map to discrete output-classes that indicate if the stock-return was a gain or loss. This is a so-called binary classification problem, and perhaps that is easier for the Neural Network.

### Convert Output Data

First we need to convert the stock-returns into 1 for gains and 0 for losses, and then split into training- and test-sets:

In [ ]:
df_cls,df_sig_rets

In [ ]:
%%time
# Convert stock-returns to binary classes where loss=0 and gain=1
# In the previous tutorial we just used the sign directly, but it
# is easier in TensorFlow / Keras to use class-values of 0 and 1.
df_cls = (np.sign(df_sig_rets[TOTAL_RETURN_1_3Y]) + 1) * 0.5

# Split the classes into training- and test-sets.
y_train_cls = df_cls.loc[tickers_train]
y_test_cls = df_cls.loc[tickers_test]

In [ ]:
%%time
# Convert stock-returns to binary classes where loss=0 and gain=1
# In the previous tutorial we just used the sign directly, but it
# is easier in TensorFlow / Keras to use class-values of 0 and 1.
df_cls = (np.sign(df_sig_rets[TOTAL_RETURN_1_1Y]) + 1) * 0.5

# Split the classes into training- and test-sets.
y_train_cls = df_cls.loc[tickers_train]
y_test_cls = df_cls.loc[tickers_test]

In [ ]:
y_train_cls.shape,y_test_cls.shape,np.unique(df_cls)

In [ ]:
df_sig_rets[TOTAL_RETURN_1_1Y].describe()

In [ ]:
#7 classes
conditions = [
    (df_sig_rets[TOTAL_RETURN_1_3Y] < (-0.4)),
    (df_sig_rets[TOTAL_RETURN_1_3Y]>=(-0.4)) & (df_sig_rets[TOTAL_RETURN_1_3Y]<(-0.2)),
    (df_sig_rets[TOTAL_RETURN_1_3Y]>=(-0.2)) & (df_sig_rets[TOTAL_RETURN_1_3Y]<(-0.1)),
    (df_sig_rets[TOTAL_RETURN_1_3Y]>=(-0.1)) & (df_sig_rets[TOTAL_RETURN_1_3Y]<0.1),
    (df_sig_rets[TOTAL_RETURN_1_3Y]>=0.1) & (df_sig_rets[TOTAL_RETURN_1_3Y]<0.2),
    (df_sig_rets[TOTAL_RETURN_1_3Y]>=0.2) & (df_sig_rets[TOTAL_RETURN_1_3Y]<0.4),
    (df_sig_rets[TOTAL_RETURN_1_3Y]>= 0.4)]
choices = [0,1,2,3,4,5,6]
df_sig_rets['return3y'] = np.select(conditions, choices)
df_cls = df_sig_rets['return3y']
#print(df_sig_rets)
np.unique(df_cls)

In [ ]:
#3 classes
conditions = [
    (df_sig_rets[TOTAL_RETURN_1_3Y] < (-0.2)),
    (df_sig_rets[TOTAL_RETURN_1_3Y]>=(-0.2)) & (df_sig_rets[TOTAL_RETURN_1_3Y]<0.2),
    (df_sig_rets[TOTAL_RETURN_1_3Y]>=0.2)]
choices = [0,1,2]
df_sig_rets['return3y'] = np.select(conditions, choices)
df_cls = df_sig_rets['return3y']
#print(df_sig_rets)
np.unique(df_cls)

In [ ]:
#3 classes
conditions = [
    (df_sig_rets[TOTAL_RETURN_1_1Y] < (-0.1)),
    (df_sig_rets[TOTAL_RETURN_1_1Y]>=(-0.1)) & (df_sig_rets[TOTAL_RETURN_1_1Y]<0.2),
    (df_sig_rets[TOTAL_RETURN_1_1Y]>=0.2)]
choices = [0,1,2]
df_sig_rets['return1y'] = np.select(conditions, choices)
df_cls = df_sig_rets['return1y']
#print(df_sig_rets)
np.unique(df_cls)

In [ ]:
np.unique(np.select(conditions, choices))

In [ ]:
# Split the classes into training- and test-sets.
y_train_cls = df_cls.loc[tickers_train]
y_test_cls = df_cls.loc[tickers_test]

In [ ]:
df_cls

In [ ]:
np.unique(df_sig_rets['return3y']), np.unique(y_train_cls)

In [ ]:
np.unique(df_sig_rets['return1y'])

In [ ]:
df_cls

In [ ]:
y_train_cls

Because this is a binary classification problem (the result is either a gain or a loss), we could build a Neural Network to use `y_train_cls` directly. But instead we want to build a more general type of Neural Network which supports any number of output classes. For this we must convert the output to so-called one-hot encoded arrays, so a class of 0 (loss) becomes the array `[1,0]` and a class of 1 (gain) becomes the array `[0,1]`.

In [ ]:
to_categorical?

In [ ]:
# Convert classes to one-hot-encoded arrays.
y_train_one_hot = to_categorical(y=y_train_cls.values,
                                 num_classes=3)

# Show the result.
y_train_one_hot

The training-set is also biased because it contains many more samples with gains compared to losses:

In [ ]:
# Fraction of the training-set that is classified as gains.
gain_fraction = (y_train_cls == 1.0).sum() / len(y_train_cls)
loss_fraction = 1.0 - gain_fraction

# Show it.
gain_fraction

When we train the Neural Network further below, we need to provide new class-weights to counter this imbalance between the two classes:

In [ ]:
y_train_cls[y_train_cls==0]

In [ ]:
compute_class_weight?

In [ ]:
# Calculate the class-weights.
class_weight = compute_class_weight('balanced',
                                    y=y_train_cls,
                                    classes=[0,1,2])

# Show them.
class_weight

We can check that when we multiply these weights by the class-fractions for losses and gains, they both end up with the same weights:

In [ ]:
class_weight * [loss_fraction, gain_fraction]

In [ ]:
# The problem is that the sklearn API returns a numpy array but the keras requires a dictionary as an input for class_weight (see here). 
# https://stackoverflow.com/questions/61261907/on-colab-class-weight-is-causing-a-valueerror-the-truth-value-of-an-array-wit
class_weight=dict(enumerate(class_weight.flatten(), 0))

In [ ]:
class_weight

### Build and Train Neural Network: Classification

Now that we have prepared the classification data, we can proceed to build the Neural Network. We will use the same overall design as for the regression model above, with minor changes for the classification task.

In [ ]:
X_train.values,y_train.values

In [ ]:
fit_args = \
{
    # For efficiency, the model is trained on batches of data.
    'batch_size': 4096,
    
    # Number of iterations aka. epochs over the training-set.
    'epochs': 20,
    
    # Fraction of the training-set used for validation after
    # each training-epoch, to assess how well the model performs
    # on unseen data.
    'validation_split': 0.2,

    # Show status during training.
    'verbose': 1,
}

In [ ]:
# The activation function applied after each layer.
# It is important to use a non-linear function, otherwise
# the Neural Network cannot learn non-linear mappings.
activation = 'relu'
#activation = 'selu'

In [ ]:
# This model reaches 0.53-0.55 of validation accuracy at epoch=20

# Create a new Keras model.
model_clf = keras.Sequential()

# Add an input layer for the signals.
# Note how we set the dimensionality of the input here.
model_clf.add(InputLayer(input_shape=(num_signals,)))

#using LeakyReLU
from keras.layers import LeakyReLU
# here change your line to leave out an activation 
model_clf.add(Dense(256))
# now add a ReLU layer explicitly:
model_clf.add(LeakyReLU(alpha=0.05))
#model_clf.add(Dropout(0.2))

# add dropout helps
from keras.layers import Dropout
# Add several dense aka. fully-connected layers.
# You can experiment with different designs.
model_clf.add(Dense(128, activation=activation))
model_clf.add(Dropout(0.3))
model_clf.add(Dense(64, activation=activation))
model_clf.add(Dropout(0.3))
model_clf.add(Dense(32, activation=activation))
model_clf.add(Dropout(0.2))
model_clf.add(Dense(16, activation=activation))
model_clf.add(Dropout(0.2))
model_clf.add(Dense(8, activation=activation))
#model_clf.add(Dropout(0.2))

# Add a layer for the output of the Neural Network.
# This has 2 outputs, one for each possible class, and it uses
# the softmax activation function for classification.
model_clf.add(Dense(3, activation='softmax'))

# Compile the model but don't train it yet.
# Note that it uses a different loss-function and metric
# than the regression-model above.
model_clf.compile(loss='categorical_crossentropy',
                  metrics=['accuracy'],
                  optimizer=RMSprop(0.001))

# Show the model.
model_clf.summary()

In [ ]:
# Create a new Keras model.
model_clf = keras.Sequential()

# Add an input layer for the signals.
# Note how we set the dimensionality of the input here.
model_clf.add(InputLayer(input_shape=(num_signals,)))

#using LeakyReLU
from keras.layers import LeakyReLU
# here change your line to leave out an activation 
model_clf.add(Dense(128))
# # now add a ReLU layer explicitly:
model_clf.add(LeakyReLU(alpha=0.05))
# model_clf.add(Dropout(0.2))

# add dropout helps
from keras.layers import Dropout
# Add several dense aka. fully-connected layers.
# You can experiment with different designs.
model_clf.add(Dense(128, activation=activation))
model_clf.add(Dropout(0.2))
model_clf.add(Dense(64, activation=activation))
model_clf.add(Dropout(0.2))
model_clf.add(Dense(32, activation=activation))
# model_clf.add(Dropout(0.2))
model_clf.add(Dense(16, activation=activation))
# model_clf.add(Dropout(0.2))
model_clf.add(Dense(8, activation=activation))
#model_clf.add(Dropout(0.2))

# Add a layer for the output of the Neural Network.
# This has 2 outputs, one for each possible class, and it uses
# the softmax activation function for classification.
model_clf.add(Dense(3, activation='softmax'))

# Compile the model but don't train it yet.
# Note that it uses a different loss-function and metric
# than the regression-model above.
model_clf.compile(loss='categorical_crossentropy',
                  metrics=['accuracy'],
                  optimizer=RMSprop(0.001))

# Show the model.
model_clf.summary()

We can now train the model. This takes a minute or two on an ordinary computer:

In [ ]:
%%time
history_clf = model_clf.fit(x=X_train.values, y=y_train_one_hot,
                            class_weight=class_weight, **fit_args)

We can then plot the training-progress as the classification accuracy which is supposed to be maximized. After 40 epochs through the training-set, the model has learned to map from signals to stock gain/loss for the training-set, with an accuracy of about 96% (this is not for the entire training-set, but only for one or more batches of data).

Unfortunately, the performance on the validation-set gets worse during optimization and ends with an accuracy of about 60%, so the model is clearly not learning the underlying relation between signals and stock gain/loss, but it has instead learned peculiarities and noise in the training-data.

In [ ]:
# Convert training history to a DataFrame.
df = pd.DataFrame(history_clf.history)

# Rename the relevant columns.
names = {'accuracy': 'Training Accuracy',
         'val_accuracy': 'Validation Accuracy'}
df = df[['accuracy', 'val_accuracy']].rename(columns=names)

# Plot the results.
df.plot()

In [ ]:
history_clf

In [ ]:
pd.DataFrame(history_clf.history)

### Classification Results

We can then use the fitted model to predict the classes for the training- and test-sets:

In [ ]:
%%time
y_train_pred_cls = model_clf.predict(X_train.values)
y_test_pred_cls = model_clf.predict(X_test.values)

The Neural Network has 2 outputs which is a likelihood score for each of the two classes:

In [ ]:
y_test_pred_cls

In [ ]:
np.argmax(y_test_pred_cls, axis=1)

Let us convert this to a single integer class-number, by taking the so-called argmax, which is the index of the largest element in each row:

In [ ]:
# Convert one-hot-encoded arrays to integer class-numbers.
y_train_pred_cls = y_train_pred_cls.argmax(axis=1)
y_test_pred_cls = y_test_pred_cls.argmax(axis=1)

# Show the result.
y_test_pred_cls

In [ ]:
np.unique(y_test_cls),np.unique(y_test_pred_cls),np.unique(y_train_one_hot)

In [ ]:
np.unique(y_test_pred_cls),np.unique(y_train_cls),np.unique(y_test_pred_cls)

We can now calculate the classification accuracy for the entire training-set, which is almost 90%, so the model has learned to map financial signals to gain/loss classifications for 1-3 year investment periods:

In [ ]:
accuracy_score(y_true=y_train_cls, y_pred=y_train_pred_cls)

Unfortunately this performance does not generalize to the test-set, which the model has not seen during training. Here the classification accuracy is much lower:

In [ ]:
accuracy_score(y_true=y_test_cls, y_pred=y_test_pred_cls)

This is even worse than simply guessing that the stock-returns are always gains, which would give a classification accuracy of about 70%:

In [ ]:
# Fraction of the test-set that is classified as gains.
(y_test_cls == 2).sum() / len(y_test_cls)

### Confusion Matrix

We can also plot a so-called Confusion Matrix, which shows how well the classes were predicted. First we need a small helper-function for making the plot:

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    """
    Plot a classification confusion matrix.
    
    :param y_true: Array of true classes.
    :param y_pred: Array of predicted classes.
    """
    
    # Class labels.
    labels = [1.0, 0.0]
    labels_text = ['Gain', 'Loss']

    # Create confusion matrix.
    mat = confusion_matrix(y_true=y_true, y_pred=y_pred,
                           labels=labels)
    
    # Normalize so all matrix entries sum to 1.0
    mat = mat / len(y_true)
    
    # Plot the matrix as a heatmap.
    sns.heatmap(mat, robust=True,
                xticklabels=labels_text,
                yticklabels=labels_text)

In [ ]:
# we now have 5 classes
def plot_confusion_matrix(y_true, y_pred):
    """
    Plot a classification confusion matrix.
    
    :param y_true: Array of true classes.
    :param y_pred: Array of predicted classes.
    """
    
    # Class labels.
    labels = [0,1,2,3,4,5,6]
    labels_text = ['bigerloss','bigloss', 'Loss','nochange',"gain","biggain","biggergain"]

    # Create confusion matrix.
    mat = confusion_matrix(y_true=y_true, y_pred=y_pred,
                           labels=labels)
    
    # Normalize so all matrix entries sum to 1.0
    mat = mat / len(y_true)
    
    # Plot the matrix as a heatmap.
    sns.heatmap(mat, robust=True,
                xticklabels=labels_text,
                yticklabels=labels_text)

In [ ]:
#  3 classes
def plot_confusion_matrix(y_true, y_pred):
    """
    Plot a classification confusion matrix.
    
    :param y_true: Array of true classes.
    :param y_pred: Array of predicted classes.
    """
    
    # Class labels.
    labels = [0,1,2]
    labels_text = ['Loss','nochange',"gain"]

    # Create confusion matrix.
    mat = confusion_matrix(y_true=y_true, y_pred=y_pred,
                           labels=labels)
    
    # Normalize so all matrix entries sum to 1.0
    mat = mat / len(y_true)
    
    # Plot the matrix as a heatmap.
    sns.heatmap(mat, robust=True,
                xticklabels=labels_text,
                yticklabels=labels_text)

For the training-set, we again see that the model has learned to accurately classify gains and losses. Note how the colours are different, because the training-set contains many more gains than losses.

In [ ]:
y_train_cls,y_train_pred_cls

In [ ]:
plot_confusion_matrix(y_true=y_train_cls, y_pred=y_train_pred_cls)

The confusion matrix for the test-set, shows that many classes were correctly predicted as gains, and there were a lot of errors in the rest of the predictions:

In [ ]:
plot_confusion_matrix(y_true=y_test_cls, y_pred=y_test_pred_cls)

### Classification Examples

We can also try and plot the model's predicted versus actual gain/loss classification for a stock in the training-set. As we can see, the model has learned this mapping quite well:

In [ ]:
stock_index=np.where(tickers_train=='IBM')
print(stock_index)

In [ ]:
# Create a DataFrame with actual and predicted classifications.
# This is for the training-set.
df_y_train_cls = pd.DataFrame(y_train_cls)
df_y_train_cls[TOTAL_RETURN_PRED] = y_train_pred_cls

# Plot the actual and predicted stock-returns for the first stock.
ticker = tickers_train[0]
_ = df_y_train_cls.loc[ticker].plot(title=ticker)

In [ ]:
df_y_train_cls.loc[ticker]

Now let us try and plot the model's predicted versus actual gain/loss classifications for the first stock in the test-set. As we can see, the model is quite bad at predicting this for data that it has not seen during its training:

In [ ]:
# Create a DataFrame with actual and predicted classifications.
# This is for the test-set.
df_y_test_cls = pd.DataFrame(y_test_cls)
df_y_test_cls[TOTAL_RETURN_PRED] = y_test_pred_cls

# Plot the actual and predicted stock-returns for the first stock.
ticker = tickers_test[0]
_ = df_y_test_cls.loc[ticker].plot(title=ticker)

In [ ]:
df_y_test_cls.loc[ticker]

## Discussion

We used a Neural Network to learn the relation between signals and future stock-returns. This worked reasonably well on the training-data, but it did not generalize to the test-data, where the Neural Network performed worse than just guessing. In fact, the result was even worse than the previous tutorial which used a Machine Learning (ML) model called a Random Forest.

The problem might be the design of the Neural Network, that is, the number and size of the layers, their activation functions, etc. You can try and use so-called Hyper-Parameter Optimization such as the [Keras Tuner](https://keras-team.github.io/keras-tuner/), to try and find a better design for the Neural Network.

But we have now seen two very different ML models having the same problem with this dataset, so perhaps **it means that these financial signals simply cannot be used to predict future stock-returns for 1-3 year investment periods.**


### Machine Learning vs. Good Science

In the previous tutorial, we had a brief discussion about Machine Learning versus Good Science. We made an important point that is worth repeating here.

In general, these experiments show that Machine Learning cannot replace good science. *Even if the ML model had performed well on the test-data, it would not have provided an explanation for the underlying relation between signals and stock-returns, so we would not know whether it was truly something that would work in the future. To explain this we need proper scientific reasoning.*

A good example of scientific reasoning is the discovery and formulation of the Law of Gravity. A simple experiment would be to drop various items from different heights and take lots of measurements of how fast the items fall to the ground.

You can then make a noisy scatter-plot that shows the relation between item-size, item-weight, drop-height, and drop-speed. Statistical analysis would be able to give you a rough idea whether there is some relation in the data and if it is statistically significant, so the observed data is probably not just due to random chance. 

Machine Learning takes this one step further, in that it might be able to create a mathematical or computational model for the non-linear relation in the data, so you could use the ML model to predict the drop-speed if you input the other data. **But the ML model would not be able to explain *why* this relation exists in the data.** For this we need a proper theoretical framework such as Classical Mechanics in the case of the Law of Gravity.


### Predicting Long-Term Stock Returns

When it comes to predicting long-term stock-returns, there are actually only 3 factors that matter: <font color='red'> **The future dividends, sales-growth, and change in the P/Sales valuation ratio.** </font>(You could instead use earnings-growth or book-value growth, and the P/E or P/Book valuation ratios.) If we can predict the future dividend, sales-growth and P/Sales ratio, then we can predict the future stock-return.

When making scatter-plots of historical P/Sales ratios versus long-term stock-returns, we often see a particular downwards-sloping curve, and it turns out there is a certain formula for this curve, which is derived from the mathematical definition of Annualized Return, see [here](https://github.com/Hvass-Labs/FinanceOps/blob/master/01C_Theory_of_Long-Term_Stock_Forecasting.ipynb). The formula takes as input the average dividend yield, sales-growth and P/Sales ratios that you predict for the future, and produces a curve that can be used to predict future stock-returns from these 3 factors and the P/Sales ratio today. The accuracy of the prediction depends on how well you have predicted these 3 factors.

Machine Learning cannot uncover and explain such relations, whether it is the Law of Gravity, or the formula for long-term stock-returns. Human reasoning is still needed for this.

## License (MIT)

This is published under the
[MIT License](https://github.com/simfin/simfin-tutorials/blob/master/LICENSE.txt)
which allows very broad use for both academic and commercial purposes.

You are very welcome to modify and use this source-code in your own project. Please keep a link to the [original repository](https://github.com/simfin/simfin-tutorials).
